In [1]:
from stable_baselines3.sac.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import SAC
import logging
import gym
import numpy as np

libs = ["../../OpenIPSL-1.5.0/OpenIPSL/package.mo",
        "../../OpenIPSL-1.5.0/ApplicationExamples/KundurSMIB/package.mo"]
mo_name = "KundurSMIB.SMIB_vref"
env_entry_point = 'examples:DymSMIBEnv'

v_ref = 1
time_step = 1
positive_reward = 1
negative_reward = -100
log_level = logging.DEBUG

config = {
    'mo_name': mo_name,
    'libs': libs,
    'v_ref': v_ref,
    'time_step': time_step,
    'positive_reward': positive_reward,
    'negative_reward': negative_reward,
    'log_level': log_level
}

from gym.envs.registration import register
env_name = "MicrogridEnv-v0"

register(
    id=env_name,
    entry_point=env_entry_point,
    kwargs=config
)

In [2]:
env = gym.make(env_name)
# env.dymola.cd(os.path.join(os.getcwd())
model = SAC(MlpPolicy, env, verbose=1, tensorboard_log="tensorboard_logs")

[True, [0.0004939890731558813]]
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\Aisling\.conda\envs\myenv\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
env.action

[]

In [4]:


min_reward = np.inf
max_reward = -np.inf
avg_reward = 0
obs = env.reset()
for _ in range(20):
    action = [1.0]
    my_action = np.multiply(0.05,action) + 1
    obs, reward, done, info = env.step(my_action)
    if done:
        env.reset()
    avg_reward += 1/30 * reward
    if reward < min_reward:
        min_reward = reward
    if reward > max_reward:
        max_reward = reward

ERROR:root:List of values for model inputs should be of the length 0,equal to the number of model inputs. Actual length 1


[True, [0.0004939890731558813]]


ValueError: List of values for model inputs should be of the length 0,equal to the number of model inputs. Actual length 1

In [ ]:
model = SAC(MlpPolicy, env, verbose=1, tensorboard_log="tensorboard_logs")

obs = env.reset()
rl_reward = 0
for _ in range(20):
    action, _state = model.predict(obs)
    my_action = np.multiply(0.05,action) + 1
    obs, reward, done, info = env.step(my_action)
    if done:
        env.reset()
    rl_reward += reward

In [ ]:
env.dymola.getLastErrorLog()

In [ ]:
env.dymola.simulateExtendedModel(env.model_name, startTime=0,
                                                stopTime=1,
                                                 initialNames=['v_ref'],
                                                 initialValues=[1.0],
                                                finalNames=env.model_output_names)

In [ ]:
env.reset()